# Coalescence time estimates between sequences with SpeedDate and BioJulia

## Introduction

SpeedDate is the name given to a method of estimating a coalescence time between two DNA sequence regions that was first implemented in the R package [HybridCheck](https://github.com/Ward9250/HybridCheck) in order to date regions of introgression in large sequence contigs. The current SpeedDate is implemented in the [SpeedDate.jl](https://github.com/Ward9250/SpeedDate.jl) package, which depends on [Bio.jl](https://github.com/BioJulia/Bio.jl) for core functionality like the input and output of DNA sequences and data, and computation of mutation counts and genetic distances.

## The dating estimation method

The coalescence time is estimated using the number of mutations that have occurred between two aligned sequences.
The calculation uses a strict molecular clock which assumes a constant substitution rate, both through time and across taxa.

Modelling the mutation accumulation process as a Bernoulli trial, the probability of observing $k$ or fewer mutations between two sequences of length $n$ can be given as:

\begin{equation}
Pr(X \le k) = \sum_{i=0}^{\lfloor k \rfloor} \binom{n}{i} p^i (1 - p)^{n-i}
\end{equation}

Where $p$ is the probability of observing a single mutation between the two aligned sequences.
The value of $p$ depends on two key factors: the substitution rate and the coalescence time.
If you assume a molecular clock, whereby two DNA sequences are both accumulating mutations at a rate $\mu$ for $t$ generations, then you may define $p = 2\mu t$.

Using these assumptions, the SpeedDate method finds the root of the following formula for $Pr(X \le k) = 0.05$, $0.5$, and $0.95$, and then divides the three answers by twice the assumed substitution rate.

\begin{equation}
f(n, k, 2\mu t, Pr(X \le k) = \left( \sum_{i=0}^{\lfloor k \rfloor} \binom{n}{i} {2\mu t}^i (1 - 2\mu t)^{n-i}   \right) - Pr(X \le k)
\end{equation}

This results in an upper, middle, and lower estimate of the coalescence time $t$ of the two sequences (expressed as the number of generations).

This method has been written into the Phylo module of the flagship package of the BioJulia project, Bio.jl as a function called `coaltime`, and SpeedDate simply bundles the steps of loading data, conting mutations in sequences, and passing that information to `coaltime` easier.

SpeedDate can be used in three forms: If you are comfortable downloading your own julia packages, there is a command line interface and a GTK based GUI bundled in [SpeedDate.jl](https://github.com/Ward9250/SpeedDate.jl), but actually since this notebook is hosted using the [Binder](http://mybinder.org/) system, you can use this notebook to use the SpeedDate method right away, no need to install anything new on your computer. 

## Let's use SpeedDate!

**NOTE!** If you modify this notebook and use it with your own data, and want store it for reference or future use, please remember to download it on your own computer, as the online session that you're using is not permanent.

With that out of the way, these packages are required to get you going:

In [1]:
using Reactive, Interact, SpeedDate


In [9]:
## ENTER THE MUTATION MODEL YOU WANT IN THIS DROPDOWN BOX - DEFAULT IS JUKES CANTOR 69.
mutation_model = dropdown(["JukesCantor69", "Kimura80"])

Interact.Options{:Dropdown,Any}(Signal{Any}(JukesCantor69, nactions=0),"","JukesCantor69","JukesCantor69",Interact.OptionDict(Any["JukesCantor69","Kimura80"],Dict{Any,Any}(Pair{Any,Any}("JukesCantor69","JukesCantor69"),Pair{Any,Any}("Kimura80","Kimura80"))),Any[],Any[])

In [15]:
## ENTER YOUR ASSUMED MUTATION RATE IN THE BOX BELOW
mutation_rate = textbox("10e-9", label = "Mutation Rate")

Interact.Textbox{String}(Signal{String}(10e-9, nactions=0),"Mutation Rate",nothing,"10e-9")

In [14]:
## WHETHER OR NOT YOU WANT TO USE SLIDING WINDOWS.
use_windows = checkbox(false, label = "Sliding Windows")

Interact.Checkbox(Signal{Bool}(false, nactions=0),"Sliding Windows",false)

In [16]:
## IF YOU ARE USING SLIDING WINDOWS, WHAT IS THE WINDOW SIZE??
window_size = textbox("100", label = "Window Size")

Interact.Textbox{String}(Signal{String}(100, nactions=0),"Window Size",nothing,"100")

In [17]:
## IF YOU ARE USING SLIDING WINDOWS, WHAT IS THE WINDOW STEP??
window_step = textbox("100", label = "Window Step")

Interact.Textbox{String}(Signal{String}(100, nactions=0),"Window Step",nothing,"100")